# Hidden Markov Model

In [2]:
import collections
import itertools
import random
import math
from sys import float_info
from collections import defaultdict
from pathlib import Path
from operator import itemgetter
from pprint import pprint

## Constants

In [3]:
DATA_ROOT = Path("data")
DATASETS = ["SG", "CN", "EN", "AL"]

## Helper Functions

In [4]:
def load_dataset(path, split=True, shuffle=False):
    """
    Load a dataset from a specified path.
    
    Args:
        path: The path to read the data from
        split (bool): Whether to split labels from each line of data
    """
    with open(path) as f:
        sequences = [sent.split("\n") for sent in f.read().split("\n\n")][:-1]
    if shuffle:
        random.shuffle(sequences)
    if split:
        sequences = [[pair.split() for pair in seq] for seq in sequences]
        sequences = [[[pair[i] for pair in s] for s in sequences] for i in [0, 1]]
    return sequences


def pairwise(sequence, include_start_stop=True):
    """
    Rolling window over iterable (with offset=1 and window_size=2)

    Args:
        sequence: The iterable to window over
        include_start_stop (bool): If True, adds START & STOP are added to either end of output
        
    Examples:
        >>> pairwise([1, 2, 3], include_start_stop=True)
        [("START", 1), (1, 2), (2, 3), (3, "STOP")]

        >>> pairwise([1, 2, 3, 4], include_start_stop=False)
        [(1, 2), (2, 3), (3, 4)]
    """
    a, b = itertools.tee(sequence)
    next(b)
    pairs = zip(a, b)
    if include_start_stop:
        pairs = itertools.chain([("START", sequence[0])], pairs, [(sequence[-1], "STOP")])
    return pairs


def flatten(sequences):
    """
    Flatten a nested sequence
    """
    return itertools.chain.from_iterable(sequences)


def unique(sequences, sort=True):
    items = set(flatten(sequences))
    if sort:
        items = sorted(items)
    return items


def count(sequences, as_probability=False):
    """
    Get a dictionary of word-count pairs in a dataset.

    Args:
        sequences: The sequence (or collection of sequences) of words to count
        as_probability (bool): Whether to return the counts as probabilties (over the entire dataset)
    """
    counts = dict(collections.Counter(flatten(sequences)))
    if as_probability:
        counts = {k: v / sum(counts.values()) for k, v in counts.items()}
    return counts


def smooth(inputs, thresh):
    """
    Replace tokens appearing less than `thresh` times with a "#UNK#" token.

    Args:
        inputs: The collection of sequences to smooth
        thresh (bool): The minimum number of occurrences required for a word to not be replaced
    """
    inputs = list(inputs)
    to_replace = {k for k, v in count(inputs, as_probability=False).items() if v < thresh}
    return [["#UNK#" if x in to_replace else x for x in sub] for sub in inputs]


def smooth_dev(sequences, train_sequences):
    """
    For each token in the given inputs, replace it with "#UNK#" if it doesn't appear in the training corpus.
    """
    train_sequences = unique(train_sequences, sort=False)
    return [[x if x in train_sequences else "#UNK#" for x in sequence] for sequence in sequences]


def get_token_map(sequences):
    """
    Get token_to_id and id_to_token maps from a collection of sequences
    """
    tokens = unique(sequences)
    return {token: i for i, token in enumerate(tokens)}


def encode_numeric(sequences, token_map=None):
    """
    Encode a collection of token sequences as numerical values
    """
    token_map = token_map or get_token_map(sequences)  # Compute token map if not provided
    return [[token_map[token] for token in sequence] for sequence in sequences], token_map


def decode_numeric(sequences, token_map):
    """
    Decode a collection of token ID sequences to tokens
    """
    token_map = {token: i for i, token in token_map.items()}  # Reverse token map
    return [[token_map[val] for val in sequence] for sequence in sequences]


def pprint_dict(d, max_entires=40):
    pprint(dict(itertools.islice(d.items(), max_entires)))

## Part 3

### Emission Parameters

In [5]:
def get_emission_parameters(observations, states):
    """
    Estimate emission paramters from a collection of observation-state pairs
    Outer: State 
    Inner: Observation
    """
    n_observations = max(flatten(observations)) + 1  # Observation space size
    n_states = max(flatten(states)) + 1  # State space size
    emission_matrix = [[0 for _ in range(n_observations)] for _ in range(n_states)]

    for state, obs in zip(states, observations):
        for s, o in zip(state, obs):
            emission_matrix[s][o] += 1

    for i in range(n_states):
        row_sum = sum(emission_matrix[i])
        for j in range(n_observations):
            emission_matrix[i][j] /= row_sum

    return emission_matrix

In [6]:
for dataset in DATASETS:
    print(f"Dataset: {dataset}")
    features, labels = load_dataset(f"data/{dataset}/train")
    features = smooth(features, 3)  # Input feature smoothing

    # Numerically encode dataset
    feature_ids, feature_map = encode_numeric(features)
    label_ids, label_map = encode_numeric(labels)

    # Calculate Emission Parameters
    emission_matrix = get_emission_parameters(feature_ids, label_ids)
    print(f"Emission Matrix: ({len(emission_matrix)} States -> {len(emission_matrix[0])} Observations)")

Dataset: SG
Emission Matrix: (7 States -> 10733 Observations)
Dataset: CN
Emission Matrix: (7 States -> 7364 Observations)
Dataset: EN
Emission Matrix: (21 States -> 6187 Observations)
Dataset: AL
Emission Matrix: (42 States -> 2698 Observations)


### Transition Parameters

In [7]:
def get_transition_parameters(state_sequences):
    """
    Estimate transition paramters from a collection of state sequences
    """
    n_states = max(flatten(state_sequences)) + 1  # State space size (Excluding START and STOP)
    transition_matrix = [[0 for _ in range(n_states + 2)] for _ in range(n_states + 1)]  # Transition matrix does not include (STOP -> other) transitions
    # transition_matrix[0] represents q(Y_i=S | Y_i-1=START)
    # transition_matrix[-1] represents q(Y_i=STOP | Y_i-1=S)
    for states in state_sequences:
        for i in range(len(states) - 1):
            if i == 0:
                transition_matrix[0][states[1] + 1] += 1
            elif i == len(states) - 2:
                transition_matrix[states[-1] + 1][-1] += 1
            else:
                transition_matrix[states[i] + 1][states[i + 1] + 1] += 1
    for row in transition_matrix:
        row_sum = sum(row)
        if row_sum:
            for i in range(len(row)):
                row[i] /= row_sum
    return transition_matrix

In [8]:
for dataset in DATASETS:
    print(f"Dataset: {dataset}")
    _, labels = load_dataset(f"data/{dataset}/train")  # Load dataset
    label_ids, label_map = encode_numeric(labels)  # Numerically encode dataset
    transition_matrix = get_transition_parameters(label_ids)
    print(f"Transition Parameters: ({len(transition_matrix)} States -> {len(transition_matrix[0])} States)")
#     print(transition_matrix)

Dataset: SG
Transition Parameters: (8 States -> 9 States)
Dataset: CN
Transition Parameters: (8 States -> 9 States)
Dataset: EN
Transition Parameters: (22 States -> 23 States)
Dataset: AL
Transition Parameters: (43 States -> 44 States)


### Viterbi Algorithm

In [16]:
%debug

> <ipython-input-4-e55607f3f341>(15)<listcomp>()
     13     if split:
     14         sequences = [[pair.split() for pair in seq] for seq in sequences]
---> 15         sequences = [[[pair[i] for pair in s] for s in sequences] for i in [0, 1]]
     16     return sequences
     17 

ipdb> sequences
*** NameError: name 'sequences' is not defined
ipdb> pair
['b']
ipdb> up
> <ipython-input-4-e55607f3f341>(15)<listcomp>()
     13     if split:
     14         sequences = [[pair.split() for pair in seq] for seq in sequences]
---> 15         sequences = [[[pair[i] for pair in s] for s in sequences] for i in [0, 1]]
     16     return sequences
     17 

ipdb> up
> <ipython-input-4-e55607f3f341>(15)<listcomp>()
     13     if split:
     14         sequences = [[pair.split() for pair in seq] for seq in sequences]
---> 15         sequences = [[[pair[i] for pair in s] for s in sequences] for i in [0, 1]]
     16     return sequences
     17 

ipdb> f
*** NameError: name 'f' is not defined
ipdb> 

In [25]:
def test_viterbi():
    features, labels = load_dataset("data/hw4/train")

    # Numerically encode dataset
    feature_ids, feature_map = encode_numeric(features)
    label_ids, label_map = encode_numeric(labels)

    # Get HMM model parameters
    emission_matrix = get_emission_parameters(feature_ids, label_ids)
    transition_matrix = get_transition_parameters(label_ids)
    
    print(emission_matrix)
    print(transition_matrix)

    # Load test dataset, numerically encode it
    dev_features = load_dataset(f"data/hw4/test", split=False)
    dev_feature_ids, _ = encode_numeric(dev_features, token_map=feature_map)  # Make sure to reuse the same token map as the training set

    # Run Viterbi algorithm to get most likely labels
    index = 0
    V = viterbi_tabulate(dev_feature_ids[index], transition_matrix, emission_matrix)
    predicted_dev_labels = viterbi_solve(V)
    print(decode_numeric([dev_feature_ids[index]], feature_map)[0])
    print(decode_numeric([predicted_dev_labels], label_map)[0])
test_viterbi()

[[0.16666666666666666, 0.5, 0.3333333333333333], [0.3333333333333333, 0.0, 0.6666666666666666], [0.16666666666666666, 0.3333333333333333, 0.5]]
[[0.0, 0.4, 0.4, 0.2, 0.0], [0.0, 0.0, 0.0, 0.5, 0.5], [0.0, 0.16666666666666666, 0.0, 0.16666666666666666, 0.6666666666666666], [0, 0, 0, 0, 0]]
['b', 'b']
['Z', 'X']


In [23]:
def log(x):
    return math.log(x) if x else -100

def viterbi_tabulate(observations, transitions, emissions):
    start_p = transitions.pop(0)

    n_states = len(transitions)
    n_obs = len(observations)
    states = tuple(range(n_states))
    
    # Default values for viterbi-table
    # The first item in the innermost list is the actual log-score.
    # The second is the "winning" state that created this score.
    V = [[[-float('inf'), None]
            for state in states]
            for _ in range(n_obs)]
    
    # First observation
    for state in states:
        V[0][state] = [
            log(start_p[state]) + log(emissions[state][observations[0]]),
            None
        ]
    
    for t in range(1, n_obs):
        for state in states:
            max_tr_prob = V[t - 1][states[0]][0] \
                          + log(transitions[states[0]][state])
            
            prev_state_selected = states[0]
            
            for prev_state in states[1:]:
                tr_prob = V[t - 1][prev_state][0] \
                          + log(transitions[prev_state][state])
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_state_selected = prev_state
            
            max_prob = max_tr_prob + log(emissions[state][observations[t]])
            
            # V[t][state] = [max_prob, prev_state_selected]
            V[t][state] = max(
                [V[t - 1][y0][0]
                 + log(transitions[y0][state])
                 + log(emissions[state][observations[t]]), 
                y0]
                for y0 in states)
    
    return V

def viterbi_solve(V):
    opt = []
    max_prob = max(value[0] for value in V[-1])
    prev = None
    for state, value in enumerate(V[-1]):
        if value[0] == max_prob:
            opt.append(state)
            prev = state
            break
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][prev][1])
        prev = V[t + 1][prev][1]
    return opt
    
def viterbi(observations, transition_matrix, emission_matrix):
    V = viterbi_tabulate(observations, transition_matrix, emission_matrix)
    return viterbi_solve(V)

In [24]:
for dataset in DATASETS[:1]:
    print(f"Dataset: {dataset}")
    features, labels = load_dataset(f"data/{dataset}/train")
    smoothed_features = smooth(features, 3)  # Input feature smoothing

    # Numerically encode dataset
    feature_ids, feature_map = encode_numeric(smoothed_features)
    label_ids, label_map = encode_numeric(labels)

    # Get HMM model parameters
    emission_matrix = get_emission_parameters(feature_ids, label_ids)
    transition_matrix = get_transition_parameters(label_ids)

    # Load dev dataset, smooth and numerically encode it
    dev_features = load_dataset(f"data/{dataset}/dev.in", split=False)
    smoothed_dev_features = smooth_dev(dev_features, smoothed_features)
    dev_feature_ids, _ = encode_numeric(smoothed_dev_features, token_map=feature_map)  # Make sure to reuse the same token map as the training set

    # Run Viterbi algorithm to get most likely labels
    index = 786
    predicted_dev_labels = viterbi(dev_feature_ids[index], transition_matrix, emission_matrix)
    print(decode_numeric([dev_feature_ids[index]], feature_map)[0])
    print(decode_numeric([predicted_dev_labels], label_map)[0])

Dataset: SG
['Join', 'the', 'SONIC', 'Drive-In', 'team', '!', 'See', 'our', 'latest', '#job', 'opening', 'here', ':', '#UNK#', '#Hospitality', '#BatonRouge', ',', 'L', '…', '#UNK#']
['O', 'B-positive', 'B-positive', 'O', 'B-neutral', 'I-positive', 'O', 'B-positive', 'O', 'B-positive', 'O', 'B-positive', 'O', 'B-positive', 'O', 'B-neutral', 'I-positive', 'B-positive', 'O', 'B-positive']


## Part 5

In [68]:
import heapq

def viterbi7_tabulate(k, observations, transitions, emissions):
    start_p = transitions.pop(0)

    n_states = len(transitions)
    n_obs = len(observations)
    states = tuple(range(n_states))
    
    # Default values for viterbi-table
    # The first item in the innermost list is the actual log-score.
    # The second is the "winning" state path that created this score.
    # You need the top k instead of the top 1 now 
    V = [[[]
          for state in states]
         for _ in range(n_obs)]
    
    # Base case
    for state in states:
        # Not that many paths we can work with here
        V[0][state].append((
            # This is the score
            log(start_p[state]) + log(emissions[state][observations[0]]),
            # This is the list of states that led here.
            # Technically this should be [START]
            tuple()
        ))
    
    # For each observation...
    for t in range(1, n_obs):
        # For each state...
        for state in states:
            # A heap that holds the scores
            h = []
            
            # Grab ALL the previous score-path pairs
            for prev_state, prev_state_scores in enumerate(V[t-1]):
                for i, entry in enumerate(prev_state_scores):
                    prev_score, prev_state_path = entry
                    
                    # Calculate a new score from previous state to the current one
                    new_score = prev_score \
                        + log(transitions[prev_state][state]) \
                        + log(emissions[state][observations[t]])
                    # Also save the chain of states that generated this score
                    new_state_path = tuple(prev_state_path) + (prev_state,)
                    
                    heapq.heappush(h, (new_score, new_state_path))

            # At this point, for regular viterbi just take the max item from the heap.
            # But this isn't regular viterbi
            
            k_best = heapq.nlargest(k, h)
                        
            # V[t][state] = [max_prob, prev_state_selected]
            V[t][state] = tuple(k_best)

    return V

def viterbi7_solve(k, V, with_score=False):
    # We did a lot of the heavy lifting before
    opt = []
    out = []
    for last_state, entries in enumerate(V[-1]):
        for e in entries:
            heapq.heappush(opt, [e, last_state])
    
    for (score, state_path), last_state in heapq.nlargest(k, opt):
        if with_score:
            out.append((score, tuple(state_path) + (last_state,)))
        else:
            out.append(tuple(state_path) + (last_state,))
    
    return out
    
def viterbi7(observations, transition_matrix, emission_matrix):
    V = viterbi7_tabulate(observations, transition_matrix, emission_matrix)
    return viterbi7_solve(V)

In [71]:
def test_viterbi7():
    features, labels = load_dataset("data/hw4/train")

    # Numerically encode dataset
    feature_ids, feature_map = encode_numeric(features)
    label_ids, label_map = encode_numeric(labels)

    # Get HMM model parameters
    emission_matrix = get_emission_parameters(feature_ids, label_ids)
    transition_matrix = get_transition_parameters(label_ids)
    
    print('Emission matrix:')
    pprint(emission_matrix)
    print('\nTransition matrix:')
    pprint(transition_matrix)

    # Load test dataset, numerically encode it
    dev_features = load_dataset(f"data/hw4/test", split=False)
    print(f'\ndev_features: {dev_features}')

    dev_feature_ids, _ = encode_numeric(dev_features, token_map=feature_map)  # Make sure to reuse the same token map as the training set

    print(f'dev_feature_ids: {dev_feature_ids}')
    
    # Run Viterbi algorithm to get most likely labels
    index = 1
    k = 7
    V = viterbi7_tabulate(k, dev_feature_ids[index], transition_matrix, emission_matrix)
    print('\nk-best Viterbi table:')
    pprint(V)
    v_best = viterbi7_solve(k, V)
    print('\nViterbi solution list (with score):')
    pprint(viterbi7_solve(k, V, with_score=True))
    
    print('\nSolutions (most likely first):')
    pprint(decode_numeric(v_best, label_map))
    
test_viterbi7()

Emission matrix:
[[0.16666666666666666, 0.5, 0.3333333333333333],
 [0.3333333333333333, 0.0, 0.6666666666666666],
 [0.16666666666666666, 0.3333333333333333, 0.5]]

Transition matrix:
[[0.0, 0.4, 0.4, 0.2, 0.0],
 [0.0, 0.0, 0.0, 0.5, 0.5],
 [0.0, 0.16666666666666666, 0.0, 0.16666666666666666, 0.6666666666666666],
 [0, 0, 0, 0, 0]]

dev_features: [['b', 'b'], ['b', 'b', 'a', 'c', 'b']]
dev_feature_ids: [[1, 1], [1, 1, 0, 2, 1]]

k-best Viterbi table:
[[[(-100.69314718055995, ())],
  [(-100.91629073187416, ())],
  [(-2.0149030205422647, ())]],
 [((-102.70805020110221, (2,)),
   (-201.3862943611199, (0,)),
   (-201.6094379124341, (1,))),
  ((-202.01490302054225, (2,)),
   (-202.7080502011022, (1,)),
   (-300.69314718055995, (0,))),
  ((-103.11351530921037, (2,)),
   (-201.79175946922805, (0,)),
   (-202.01490302054225, (1,)))],
 [((-204.49980967033025, (2, 0)),
   (-204.9052747784384, (2, 2)),
   (-303.17805383034795, (0, 0)),
   (-303.40119738166214, (1, 0)),
   (-303.5835189384561, (0, 2